# **Predictive Future Sales**
**1. Exploratory Data Analysis**

**Introduction:**
In this competition you are provided with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. We are asking you to predict total sales for every product and store in the next month. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products changes every month. 
This notebook contains general information about the methods using which you can approach the problem statement.

Lets first discuss what we are given and what we have to predict. About our dataset:

The features in our *training* data:

1. date - every date of items sold
2. date_block_num - this number is given to every month
3. shop_id - unique number of every shop
4. item_id - unique number of every item
5. item_price - price of every item
6. item_cnt_day - number of items sold on a particular day

The features in our *testing* data :

1. ID - unique for every (shop_id,item_id) pair.
2. shop_id - unique number of every shop
3. item_id - unique number of every item

Daily historical sales are given *from Jan 2013 to Oct 2015*. The **task** is to predict total sales for every product and store in the next month. The **goal** is here to minimise the performance metric:*RMSE score*.

In [ ]:
#import python packages and libs#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

: 

: 

In [ ]:
#import the raw datasets#
train=pd.read_csv("/Users/macbookpro/Desktop/sales_train.csv")
items=pd.read_csv("/Users/macbookpro/Desktop/items.csv")
categories=pd.read_csv('/Users/macbookpro/Desktop/item_categories.csv')
shops=pd.read_csv('/Users/macbookpro/Desktop/shops.csv')
test=pd.read_csv('/Users/macbookpro/Desktop/test.csv')

**Dataset Overview**

Datasets apart from the test and train dataset that are given to us:

1. item_categories.csv - the item category name along with the category ID
2. items.csv - the item name along with item ID and category ID
3. shops.csv - the shop name along with shop ID

**1.1Categories**

In [ ]:
print("First 5 Entries")
print(categories.head(5))

print("Information")
print(categories.info())

print("Data Types")
print(categories.dtypes)

print("Missing Value")
print(categories.isnull().sum())

print("Null Value")
print(categories.isna().sum())

print("Shape")
print(categories.shape)

print('Description')
categories.describe()

**1.2 Items**

In [ ]:
print("First 5 Entries")
print(items.head(5))

print("Information")
print(items.info())

print("Data Types")
print(items.dtypes)

print("Missing value")
print(items.isnull().sum())

print("Null value")
print(items.isna().sum())

print("Shape of Data")
print(items.shape)

print('Description')
items.describe()

**1.3 Shops**

In [ ]:
print("First 5 Entries")
print(shops.head(5))

print("Info")
print(shops.info())

print("Data Types")
print(shops.dtypes)

print("Missing value")
print(shops.isnull().sum())

print("Null value")
print(shops.isna().sum())

print("Shape of Data")
print(shops.shape)

print('Description')
shops.describe()

**1.4 Training**

In [ ]:
print("First 5 entries")
print(train.head())

In [ ]:
print("Shape")
print(train.shape)

In [ ]:
print("Infomation")
print(train.info())

In [ ]:
print("Data Types")
print(train.dtypes)

In [ ]:
print("Missing NaN values")
print(train.isnull().sum())

In [ ]:
print("Null Values")
print(train.isna().sum())

In [ ]:
train.describe()

In [ ]:
#Change the datetime format
train['date'] = pd.to_datetime(train['date'], format = '%d.%m.%Y')
train

In [ ]:
#join the training dataset
train = train.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(categories, on='item_category_id', rsuffix='_')

In [ ]:
print('Min date from train set: %s' % train['date'].min().date())
print('Max date from train set: %s' % train['date'].max().date())

In [ ]:
#Note that this function can be modified by choosing to calculate duplicates over only a subset of features, keeping the last entry, etc.
train.drop_duplicates(inplace=True,keep='first')
print(train)

In [ ]:
#Data cleaning and remove rows with negative item price.
train = train.query('item_price > 0') 
train

**Data Leakage🌟**

When there's data leakage in the data used for machine learning model,we will get a high train and test accuracy, implying that the model is good enough for production. It will neither underfit or overfit.

However, when implementing the machine learning model in production, it will no longer be introduced to one feature. Because it is not available when you need the model’s predictions. The feature missing might even be the most important feature for determining the right class: the leaked data.

When implying the machine learning model in production, we will see that the predictions are not reliable.

We'll only be using only the "shop_id" and "item_id" that appear on the test dataset. The idea here is that we know on which shops and items we are going to predict, because of the test set, training on those rows you will get a data distribution closer to the test set, so probably this is a better idea.

In [ ]:
test_shop_ids = test['shop_id'].unique()
test_item_ids = test['item_id'].unique()
# Only shops that exist in test set.
lk_train = train[train['shop_id'].isin(test_shop_ids)]
# Only items that exist in test set.
lk_train = lk_train[lk_train['item_id'].isin(test_item_ids)]

print('Data set size before leaking:', train.shape[0])
print('Data set size after leaking:', lk_train.shape[0])

In [ ]:
#After leakage,drop the text columns since they are not significant for predictions.
lk_train

In [ ]:
lk_train.drop(['item_name','shop_name','item_category_name'],axis=1)

In [ ]:
train_monthly = lk_train[['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day','item_category_id']]
train_monthly.head()

**1.5 EDA for Item_id**
1. How the monthly sum and mean vary for with item_id for each month
2. The outdated items, over perhaps the last 6 months.

In [ ]:
train_by_item_id = train.pivot_table(index=['item_id'],values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
train_by_item_id.columns = train_by_item_id.columns.droplevel().map(str)
train_by_item_id = train_by_item_id.reset_index(drop=True).rename_axis(None, axis=1)
train_by_item_id.columns.values[0] = 'item_id'
# print(train_by_item_id.sum()[1:])
train_by_item_id.sum()[1:].plot(legend=True, label="Monthly sum")

In [ ]:
train_by_item_id.mean()[1:].plot(legend=True, label="Monthly mean")

In [ ]:
train.item_id.nunique()

In [ ]:
#outdated after month 27 in our 34 month window
outdated_items = train_by_item_id[train_by_item_id.loc[:,'27':].sum(axis=1)==0] 
print('Outdated items:', len(outdated_items))

In [ ]:
print('Outdated items in test set:', len(test[test['item_id'].isin(outdated_items['item_id'])]))

**Summary**
1. There are no missing values at all. 
2. Number of sold items usually declines over the year.
3. There are peaks in December and similar item count zig-zag behavior can be seen in June-July-August. This can be due to these periods being vacation time or possibly a national holiday.
4. 12391 of 21807 items in the train set are outdated since the last 6 months, which is quite huge.
5. For test set: 6888 outdated.

**1.6 EDA for Shop_id**

In [ ]:
train_by_shop_id = train.pivot_table(index=['shop_id'],values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
train_by_shop_id.columns = train_by_shop_id.columns.droplevel().map(str)
train_by_shop_id = train_by_shop_id.reset_index(drop=True).rename_axis(None, axis=1)
train_by_shop_id.columns.values[0] = 'shop_id'

for i in range(6,34):
    print('Does not exist in month',i,train_by_shop_id['shop_id'][train_by_shop_id.loc[:,'0':str(i)].sum(axis=1)==0].unique())

for i in range(6,28):
    print('Shop is outdated for month',i,train_by_shop_id['shop_id'][train_by_shop_id.loc[:,str(i):].sum(axis=1)==0].unique())

In [ ]:
print('Recently opened shop items:', len(test[test['shop_id']==36])) 